In [2]:
import json, gzip
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('stopwords')
import itertools
import pickle

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from gensim.models import KeyedVectors
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/briandorsey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
path = "train.jsonl.gz"
data = []

with gzip.open(path) as f:
    for ln in f:
        obj = json.loads(ln)
        data.append(obj)
#         data.append([obj['text'], obj['summary']])

In [4]:
df = pd.DataFrame(data)

In [5]:
df

,archive,compression,compression_bin,coverage,coverage_bin,date,density,density_bin,summary,text,title,url
0,http://web.archive.org/web/20060618204254id_/h...,137.470588,high,1.000000,high,20060618204254,7.823529,mixed,A surge in discriminatory behavior toward blac...,"HAMBURG, Germany, June 3  As he left the socc...",Surge in Racist Mood Raises Concerns on Eve of...,http://www.nytimes.com/2006/06/04/sports/socce...
1,http://web.archive.org/web/20060620043011id_/h...,33.636364,medium,0.909091,medium,20060620043011,4.727273,mixed,"The volume of information harvested, without \...","WASHINGTON, Dec. 23 - The National Security Ag...","Spy Agency Mined Vast Data Trove, Officials Re...",http://www.nytimes.com/2005/12/24/politics/24s...
2,http://web.archive.org/web/20060909062911id_/h...,33.880000,medium,1.000000,high,20060909062911,11.720000,extractive,The battle between Pfizer Inc.'s investors and...,IF outsized executive pay has indeed become a ...,Investors vs. Pfizer: Guess Who Has the Guns?,http://www.nytimes.com/2006/04/23/business/you...
3,http://web.archive.org/web/20080313232743id_/h...,11.894118,low,0.988235,high,20080313232743,38.988235,extractive,"If Simon Rex looks a little familiar, it may n...",BY A.J. BENZA & MICHAEL LEWITTES\n\nIf Simon R...,REX FLEXED PECS FOR SKIN PICS,http://www.nydailynews.com/archives/gossip/199...
4,http://web.archive.org/web/20080314003027id_/h...,3.932584,low,0.921348,medium,20080314003027,36.629213,extractive,POPEYE-WORTHY PIE. PHYLLO DOUGH WRAPS SPINACH ...,Spinach has terrorized generations of veggie-p...,POPEYE-WORTHY PIE. PHYLLO DOUGH WRAPS SPINACH ...,http://www.nydailynews.com/archives/entertainm...
5,http://web.archive.org/web/20080520122148id_/h...,4.597561,low,0.987805,high,20080520122148,23.475610,extractive,"All day, every day, Cheryl Bernstein thanks he...","All day, every day, Cheryl Bernstein thanks he...",JOY FOR ADDICTS ON MEND AS CHILDREN ARE RETURNED,http://www.nydailynews.com/archives/news/2001/...
6,http://web.archive.org/web/20080711053245id_/h...,20.085366,medium,0.939024,medium,20080711053245,16.890244,extractive,By JOHN MARZULLI DAILY NEWS POLICE BUREAU CHIE...,With Police Commissioner Bernard Kerik crackin...,QUICK WORK BY THE COPS NYPD response time plunges,http://www.nydailynews.com/archives/news/2001/...
7,http://web.archive.org/web/20090205180750id_/h...,4.382716,low,0.975309,high,20090205180750,22.234568,extractive,Jail inmates flout the city's newest law every...,"Wednesday, April 19th 1995, 2:35AM\n\nJail inm...",JAILED SMOKERS PUT CIG LAW TO TEST,http://www.nydailynews.com/archives/news/1995/...
8,http://web.archive.org/web/20090208012929id_/h...,18.868421,medium,0.986842,high,20090208012929,17.723684,extractive,Did Tatum O'Neal's latest battle with ex-husba...,BY GEORGE RUSH AND JOANNA MOLLOY With Kasia An...,REPORT: TATUM FIGHTS ANOTHER ROUND,http://www.nydailynews.com/archives/gossip/200...
9,http://web.archive.org/web/20090310185130id_/h...,1.752941,low,0.941176,medium,20090310185130,42.658824,extractive,COOL COCKTAIL A summer drink you'll cotton to ...,"Wednesday, May 18th 2005, 9:59AM\n\nSummer is ...",A SUMMER DRINK YOU'LL COTTON TO,http://www.nydailynews.com/archives/lifestyle/...


In [6]:
train_df = pd.DataFrame(data, columns=['text', 'summary'])

In [7]:
# stops = set(stopwords.words('english'))

def text_to_word_list(text):
    ''' Pre process and convert texts to a list of words '''
    text = str(text)
    text = text.lower()

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    text = text.split()

    return text

# used to encode words
vocabulary = {"PAD": 0, "SOS": 1, "EOS": 2, "UNK": 3} 

# used to decode (encoded) words
inverse_vocabulary = {0: "PAD", 1: "SOS", 2: "EOS", 3: "UNK"} 

# load pre-trained word embeddings
word2vec = KeyedVectors.load_word2vec_format('~/Downloads/GoogleNews-vectors-negative300.bin', binary=True)

/usr/local/anaconda3/envs/msds/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [12]:
cols = ['text', 'summary']
# Iterate over datasets, if there are multiple
for dataset in [train_df]:
    # Iterate through articles
    for index, row in dataset.iterrows():
        # Iterate through the text of both questions of the row
        for col in cols:
            # text to number representation
            t2n = [1]  
            for word in text_to_word_list(row[col]):
                # Leave out words without embeddings
                if word not in word2vec.vocab:
                    continue
                elif word not in vocabulary:
                    vocabulary[word] = len(vocabulary)
                    t2n.append(len(inverse_vocabulary))
                    inverse_vocabulary[len(inverse_vocabulary)] = word
                else:
                    t2n.append(vocabulary[word])
            t2n.append(2)
            # Replace article or summary text with number representation
            dataset.at[index, col] = t2n

In [13]:
train_df

,text,summary
0,"[1, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, ...","[1, 148, 17, 149, 150, 112, 151, 136, 60, 79, ..."
1,"[1, 742, 743, 11, 646, 307, 744, 132, 596, 745...","[1, 11, 762, 763, 764, 769, 770, 771, 272, 772..."
2,"[1, 910, 940, 840, 941, 132, 942, 569, 943, 94...","[1, 11, 954, 947, 948, 949, 950, 72, 951, 1027..."
3,"[1, 136, 1153, 1154, 910, 1155, 1156, 1157, 11...","[1, 910, 1155, 1156, 1157, 1158, 265, 167, 422..."
4,"[1, 1413, 132, 1414, 1415, 1416, 1417, 1418, 3...","[1, 1510, 1435, 1427, 1428, 1511, 1413, 17, 15..."
5,"[1, 264, 898, 1515, 898, 1516, 1517, 1518, 108...","[1, 264, 898, 1515, 898, 1516, 1517, 1518, 108..."
6,"[1, 30, 1600, 1601, 1602, 1603, 11, 1604, 11, ...","[1, 136, 1036, 1614, 1615, 1600, 1834, 970, 30..."
7,"[1, 1293, 102, 1466, 1835, 1836, 1837, 11, 20,...","[1, 1835, 1836, 1837, 11, 20, 1838, 467, 1515,..."
8,"[1, 136, 1216, 1898, 1899, 30, 1900, 1901, 960...","[1, 497, 631, 1904, 1905, 954, 30, 1906, 1528,..."
9,"[1, 1293, 422, 75, 2195, 272, 2196, 652, 8, 21...","[1, 2251, 2207, 2195, 2211, 525, 86, 2208, 191..."


In [14]:
import pickle

with open("vocab.pkl","wb") as f:
    pickle.dump(vocabulary, f)
    
with open("inv_vocab.pkl", "wb") as f:
    pickle.dump(inverse_vocabulary, f)
    
train_df.to_csv('train_df.csv', index=False)

In [49]:
text_lens = train_df.text.map(lambda x: len(x))
summary_lens = train_df.summary.map(lambda x: len(x))

In [52]:
text_lens.describe(), summary_lens.describe()

(count    995041.000000
 mean        585.993332
 std         764.028830
 min           0.000000
 25%         244.000000
 50%         465.000000
 75%         749.000000
 max      102469.000000
 Name: text, dtype: float64, count    995041.000000
 mean         23.583946
 std          23.694143
 min           0.000000
 25%          14.000000
 50%          20.000000
 75%          26.000000
 max        5676.000000
 Name: summary, dtype: float64)

In [55]:
sub_train_df = train_df[(text_lens < 750) & (summary_lens < 26)]

In [56]:
sub_text_lens = sub_train_df.text.map(lambda x: len(x))
sub_summary_lens = sub_train_df.summary.map(lambda x: len(x))

In [62]:
sub_train_df.loc[:, 'text'] = sub_train_df['text'].apply(lambda x: ((sub_text_lens.max() - len(x)) * [0]) + x)
sub_train_df.loc[:, 'summary'] = sub_train_df['summary'].apply(lambda x: x + ((sub_summary_lens.max() - len(x)) * [0]))

/usr/local/anaconda3/envs/msds/lib/python3.7/site-packages/pandas/core/indexing.py:635: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value


In [58]:
embedding_dim = 300
embeddings = 1 * np.random.randn(len(vocabulary) + 1, embedding_dim)  # This will be the embedding matrix
embeddings[0] = 0  # So that the padding will be ignored

In [64]:
sub_train_df

,text,summary
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 759, 760, 761, ..."
2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 8, 951, 944, 945, 946, 947, 69..."
12,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
14,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
20,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1154, 115, 8, 3301, 3302, 3303, 8, 5..."
23,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
24,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 981, 3674, 870,..."
35,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[380, 1589, 3158, 37, 5434, 409, 502, 228, 128..."
40,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
41,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [67]:
sub_train_df.to_csv('sub_train_df.csv', index=False)